# Lab 3: Side-Channel Attacks
In this lab we will implement a correlation power analysis to figure out the secret key used by your present implementation.

In [ ]:
import numpy as np
import chipwhisperer as cw
import utils
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
from reference import present

In [ ]:
SCOPETYPE = 'CWNANO' # or CWNANO
PLATFORM = 'CWNANO'  # or CWNANO
SS_VER="SS_VER_2_1"

In [ ]:
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial2)
prog = cw.programmers.STM32FProgrammer

In [ ]:
scope.default_setup()
scope.adc.samples = 50_000

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../hw/secure-sensor-v3/
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
cw.program_target(scope, prog, f"../hw/secure-sensor-v3/secure-sensor-{PLATFORM}.hex")

## Step 1: Taking the measurements
To perform the attack we will need two things: the power consumption measurements (traces), and the plaintexts. Fill the code cell to construct the full traces dataset.

In [ ]:
# capture single trace
scope.arm()
target.send_cmd(0x01, 0x01, bytearray([]))
scope.capture()
pl = target.simpleserial_read(cmd='s')
plt.plot(scope.get_last_trace())

In [ ]:
# Capture traces
utils.reset_target(scope)
N = 1000
traces = np.zeros((N, scope.adc.samples))
for i in tqdm.tnrange(N):
    # TODO capture trace and add to traces
    pass

In [ ]:
# Capture plaintexts
utils.reset_target(scope)
pts = np.zeros((N, 16), dtype=np.uint8)
for i in tqdm.tnrange(N):
    target.send_cmd(0x01, 0x02, bytearray([]))
    pts[i] = target.simpleserial_read(cmd='s')

## Step 2: Target Value
Now we need to figure out exactly which intermediate value we want to target (and why?), and what leakage model we want to use. We will be attacking byte index 2 (using a starting index of 0).

In [ ]:
attack_byte = 2
sbox = np.array(present.sbox)

## Step 3: Implement CPA
The last step is to implement the correlation power analysis attack, you can use the pseudocode shared in the lecture to guide you in filling out the template code below.

In [ ]:
key_vector = np.zeros(KEY_SPACE)
for kguess in tqdm.tnrange(KEY_SPACE):
    # construct the hypotheses of the leaky value based on the current key guess
    hypotheses = ...
    # calculate the correlation between your hypotheses and the traces
    correlations = ...
    # assign a single value to your key guess to estimate the likelihood of it being the correct key
    key_vector[kguess] = ...


# Disconnect

In [ ]:
target.dis()
scope.dis()